In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import praw
import re

In [2]:
reddit_1 = "https://www.reddit.com/r/IntermittentFasting/new/"
reddit_2 = "https://www.reddit.com/r/AnorexiaNervosa/new/"

In [3]:
req1 = requests.get(reddit_1)
req2 = requests.get(reddit_2)
print(req1.status_code)
print(req2.status_code)

200
200


In [4]:
soup1 = BeautifulSoup(req1.text,'lxml')
soup2 = BeautifulSoup(req2.text,'lxml')

In [5]:
redditscrapper = praw.Reddit(
    client_id='mTKAc7piwaoiD3fvkhY7qA',
    client_secret='GdT29i_cBYDTJwb0eExYEh6prVceGg',
    user_agent='Personal Scrapping App v1.0 (by /u/AlicornGaia) - Contact: weegeevirus9001@gmail.com'
)

In [6]:
# Extract subreddit name from URL using regular expression
subreddit_name_1 = re.search(r'/r/([^/]+)/new/', reddit_1).group(1)
subreddit_name_2 = re.search(r'/r/([^/]+)/new/', reddit_2).group(1)

# Print the extracted subreddit names
print(f'Subreddit 1 Name: {subreddit_name_1}')
print(f'Subreddit 2 Name: {subreddit_name_2}')

Subreddit 1 Name: IntermittentFasting
Subreddit 2 Name: AnorexiaNervosa


In [7]:
# Access the subreddit objects
subreddit_1 = redditscrapper.subreddit(subreddit_name_1)
subreddit_2 = redditscrapper.subreddit(subreddit_name_2)

# Access the new posts for each subreddit
new_posts_1 = subreddit_1.new(limit=10000)  # Change the limit as needed
new_posts_2 = subreddit_2.new(limit=10000)  # Change the limit as needed

In [8]:
# Create a dictionary to store post data
posts_dict = {
    "Title": [],
    "Post Text": [],
    "ID": [],
    "Score": [],
    "Total Comments": [],
    "Post URL": [],
    "Subreddit": []
}

# Iterate through the new posts and collect data
for post in new_posts_1:
    # Title of each post
    posts_dict["Title"].append(post.title)

    # Text inside a post
    posts_dict["Post Text"].append(post.selftext)

    # Unique ID of each post
    posts_dict["ID"].append(post.id)

    # The score of a post
    posts_dict["Score"].append(post.score)

    # Total number of comments inside the post
    posts_dict["Total Comments"].append(post.num_comments)

    # URL of each post
    posts_dict["Post URL"].append(post.url)
    
    # Subreddit of each post
    posts_dict["Subreddit"].append(post.subreddit.display_name)
    
# Iterate through the new posts for subreddit 2 and collect data
for post in new_posts_2:
    # Title of each post
    posts_dict["Title"].append(post.title)

    # Text inside a post
    posts_dict["Post Text"].append(post.selftext)

    # Unique ID of each post
    posts_dict["ID"].append(post.id)

    # The score of a post
    posts_dict["Score"].append(post.score)

    # Total number of comments inside the post
    posts_dict["Total Comments"].append(post.num_comments)

    # URL of each post
    posts_dict["Post URL"].append(post.url)
    
    # Subreddit of each post
    posts_dict["Subreddit"].append(post.subreddit.display_name)

# Saving the data in a Pandas DataFrame
all_posts = pd.DataFrame(posts_dict)
all_posts.head()

,Title,Post Text,ID,Score,Total Comments,Post URL,Subreddit
0,"New here, just curious about your opinions. Am...","Hi all!\n\nI'm a big guy, 6.4 feet, 240 pounds...",16nfts2,1,1,https://www.reddit.com/r/intermittentfasting/c...,intermittentfasting
1,IF 20:4 Working Out and Fasting Window,"Hello All,\n\nFirstly, I am very glad we have ...",16nftnf,0,0,https://www.reddit.com/r/intermittentfasting/c...,intermittentfasting
2,Who was the crazy black dude from years back?,\nDisclaimer: I’m not insinuating that intermi...,16nfn7m,0,2,https://www.reddit.com/r/intermittentfasting/c...,intermittentfasting
3,Daily Fasting Check-in!,"* **Type** of fast (water, juice, smoking, etc...",16ndjy3,2,0,https://www.reddit.com/r/intermittentfasting/c...,intermittentfasting
4,IF Blood Glucose,i’ve been IF since 2016. avg 20 ish hours a da...,16nc355,3,9,https://i.redd.it/jgaiv5uz9cpb1.jpg,intermittentfasting


In [9]:
all_posts.describe()

,Score,Total Comments
count,1988.000000,1988.000000
mean,64.339034,12.704728
std,265.310386,21.068342
min,0.000000,0.000000
25%,3.000000,3.000000
50%,7.000000,6.000000
75%,22.000000,13.000000
max,3866.000000,271.000000


In [10]:
# Define a function to check if a post is image-only
def is_image_only_post(post):
    # Check if the post's content (Post Text) is empty (no text) and the URL is an image or an image-hosting site
    return not post["Post Text"] and post["Post URL"].endswith(('.jpg', '.jpeg', '.png', '.gif', '.gifv', '.webp'))

# Apply the function to the DataFrame to create a Boolean mask
image_only_mask = all_posts.apply(is_image_only_post, axis=1)

# Filter out image-only posts
filtered_posts = all_posts[~image_only_mask]

# Now, filtered_posts contains posts that are not image-only

In [11]:
filtered_posts.head()

,Title,Post Text,ID,Score,Total Comments,Post URL,Subreddit
0,"New here, just curious about your opinions. Am...","Hi all!\n\nI'm a big guy, 6.4 feet, 240 pounds...",16nfts2,1,1,https://www.reddit.com/r/intermittentfasting/c...,intermittentfasting
1,IF 20:4 Working Out and Fasting Window,"Hello All,\n\nFirstly, I am very glad we have ...",16nftnf,0,0,https://www.reddit.com/r/intermittentfasting/c...,intermittentfasting
2,Who was the crazy black dude from years back?,\nDisclaimer: I’m not insinuating that intermi...,16nfn7m,0,2,https://www.reddit.com/r/intermittentfasting/c...,intermittentfasting
3,Daily Fasting Check-in!,"* **Type** of fast (water, juice, smoking, etc...",16ndjy3,2,0,https://www.reddit.com/r/intermittentfasting/c...,intermittentfasting
4,IF Blood Glucose,i’ve been IF since 2016. avg 20 ish hours a da...,16nc355,3,9,https://i.redd.it/jgaiv5uz9cpb1.jpg,intermittentfasting


In [12]:
print(filtered_posts.columns)

Index(['Title', 'Post Text', 'ID', 'Score', 'Total Comments', 'Post URL',
       'Subreddit'],
      dtype='object')


In [13]:
print(f'Number of rows in all_posts: {all_posts.shape[0]}')
print(f'Number of rows in filtered_posts: {filtered_posts.shape[0]}')

Number of rows in all_posts: 1988
Number of rows in filtered_posts: 1924


In [14]:
filtered_posts.to_csv("Filtered posts.csv", index=True)

In [15]:
# Set pandas display options to show the entire content of the "Post Text" column
pd.set_option('display.max_colwidth', None)

# Now, when you display the DataFrame, the "Post Text" column will show the full content
filtered_posts

,Title,Post Text,ID,Score,Total Comments,Post URL,Subreddit
0,"New here, just curious about your opinions. Am I doing this right?","Hi all!\n\nI'm a big guy, 6.4 feet, 240 pounds (I think, didn't weight myself for a few months) and done with my bad shape. My biggest problem is beers, especially in summer I drink almost daily a few beers and in the weekends even more.\n\nI started IF 1,5 week ago now and I am curious if im doing things right. \n\nI installed Myfitnesspal for counting calories and 2100 is my daily goal to lose 1 pound a week. I eat dinner at 18.00 and eat breakfast the next day between 10.00 and 11.00, so thats 16-8/17-7 right?\n\nMy daily calorie intake is 1800 - 2000 and healthy, but the weekend is tough. I have kids who like to eat fastfood once a week and besides that I had 8 beers this weekend (way less then before, but still a lot), so the weekends I get above my goal (2400 and 2600 friday and saturday).\n\nI sport like 2-3 times a week, tennis for at least 1 hour a time. \n\nSo now you know my story I have a few questions:\n\n1. Is it ok to have a few beers in the weekend? I want to lose weight but thinking about no alcohol anymore is tough, I really like my beer in the weekend (it's a bad habit, I know!). \n2. With IF 16-8 /17-7 I still have breakfast and lunch, do I need to skip a meal, or is it 'normal' to have breakfast and lunch?\n\nCurious about your thoughts and advice! \n\nCheers\n\n&#x200B;\n\n&#x200B;",16nfts2,1,1,https://www.reddit.com/r/intermittentfasting/comments/16nfts2/new_here_just_curious_about_your_opinions_am_i/,intermittentfasting
1,IF 20:4 Working Out and Fasting Window,"Hello All,\n\nFirstly, I am very glad we have a subreddit dedicated to this.\n\nI am on a OMAD 20:4 regime at the moment. \n\n* My fasting window is between 3:30 PM - 7:30 PM. \n* I work out between 5AM - 8 AM depending on my work schedule and when I start work. I usually break my fast around \n\n&#x200B;\n\n1. Is this regime fine? \n2. Does my feeding window *have* to be after my IF workout window **or** am I okay with this setup?\n\n&#x200B;\n\nI am worried I am working out and not in a fasted state suddenly even though it is normally 12 hours after my meal intake. \n\n&#x200B;\n\nAppreciate the help and feedback from you all. Thanks :)",16nftnf,0,0,https://www.reddit.com/r/intermittentfasting/comments/16nftnf/if_204_working_out_and_fasting_window/,intermittentfasting
2,Who was the crazy black dude from years back?,"\nDisclaimer: I’m not insinuating that intermittent fasting is crazy, just that there was a crazy guy who also happened to intermittent fast.\n\nI might’ve ran across this guy on YouTube in 2017. He was an older black man with a very weird manner of speaking.\n\n- Very slow cadence, choice of words was odd and deliberate\n- Says he eats nuts and berries once a day\n- Claimed to be a doctor or something…\n- might of been some stuff about anti-aging…\n\nThat’s all I can reliably remember. The guy who introduced me to him was kind of mental as well so I started getting cult vibes.\n\nOnly asking here because he presented as some sort of subject matter expert on fasting so thought he might have popped up in these circles.\n\n\nP.S. It’s really scary to post something like this in a new sub because I have no idea what I’m in for.\n\nTop 3 Predictions:\n\n- “Never heard of him…”\n- “Yeah, that guy was crazy! Here’s a link!” \n- “How dare you insult the trans-dimensional Messiah!”",16nfn7m,0,2,https://www.reddit.com/r/intermittentfasting/comments/16nfn7m/who_was_the_crazy_black_dude_from_years_back/,intermittentfasting
3,Daily Fasting Check-in!,"* **Type** of fast (water, juice, smoking, etc.)\n* **Context** of fast (start, end, day x of y, etc.)\n* **Length** of fast (8 hours, 3 days, etc.)\n* **Why?** What you hope to accomplish with your fast\n* **Notes** How is it going so far? Any concerns? Insights to share?\n\nBe sure to check back often as comments get posted throughout the day. Sort comments by ""new"

In [22]:
# Create a new DataFrame with only the "Post Text" and "Subreddit" columns
simplified_posts = filtered_posts[["Post Text", "Subreddit"]].copy()

# Reset the index of the new DataFrame
simplified_posts.reset_index(drop=True, inplace=True)

# Now, 'simplified_posts' contains only the "Post Text" and "Post URL" columns

In [23]:
simplified_posts

,Post Text,Subreddit
0,"Hi all!\n\nI'm a big guy, 6.4 feet, 240 pounds (I think, didn't weight myself for a few months) and done with my bad shape. My biggest problem is beers, especially in summer I drink almost daily a few beers and in the weekends even more.\n\nI started IF 1,5 week ago now and I am curious if im doing things right. \n\nI installed Myfitnesspal for counting calories and 2100 is my daily goal to lose 1 pound a week. I eat dinner at 18.00 and eat breakfast the next day between 10.00 and 11.00, so thats 16-8/17-7 right?\n\nMy daily calorie intake is 1800 - 2000 and healthy, but the weekend is tough. I have kids who like to eat fastfood once a week and besides that I had 8 beers this weekend (way less then before, but still a lot), so the weekends I get above my goal (2400 and 2600 friday and saturday).\n\nI sport like 2-3 times a week, tennis for at least 1 hour a time. \n\nSo now you know my story I have a few questions:\n\n1. Is it ok to have a few beers in the weekend? I want to lose weight but thinking about no alcohol anymore is tough, I really like my beer in the weekend (it's a bad habit, I know!). \n2. With IF 16-8 /17-7 I still have breakfast and lunch, do I need to skip a meal, or is it 'normal' to have breakfast and lunch?\n\nCurious about your thoughts and advice! \n\nCheers\n\n&#x200B;\n\n&#x200B;",intermittentfasting
1,"Hello All,\n\nFirstly, I am very glad we have a subreddit dedicated to this.\n\nI am on a OMAD 20:4 regime at the moment. \n\n* My fasting window is between 3:30 PM - 7:30 PM. \n* I work out between 5AM - 8 AM depending on my work schedule and when I start work. I usually break my fast around \n\n&#x200B;\n\n1. Is this regime fine? \n2. Does my feeding window *have* to be after my IF workout window **or** am I okay with this setup?\n\n&#x200B;\n\nI am worried I am working out and not in a fasted state suddenly even though it is normally 12 hours after my meal intake. \n\n&#x200B;\n\nAppreciate the help and feedback from you all. Thanks :)",intermittentfasting
2,"\nDisclaimer: I’m not insinuating that intermittent fasting is crazy, just that there was a crazy guy who also happened to intermittent fast.\n\nI might’ve ran across this guy on YouTube in 2017. He was an older black man with a very weird manner of speaking.\n\n- Very slow cadence, choice of words was odd and deliberate\n- Says he eats nuts and berries once a day\n- Claimed to be a doctor or something…\n- might of been some stuff about anti-aging…\n\nThat’s all I can reliably remember. The guy who introduced me to him was kind of mental as well so I started getting cult vibes.\n\nOnly asking here because he presented as some sort of subject matter expert on fasting so thought he might have popped up in these circles.\n\n\nP.S. It’s really scary to post something like this in a new sub because I have no idea what I’m in for.\n\nTop 3 Predictions:\n\n- “Never heard of him…”\n- “Yeah, that guy was crazy! Here’s a link!” \n- “How dare you insult the trans-dimensional Messiah!”",intermittentfasting
3,"* **Type** of fast (water, juice, smoking, etc.)\n* **Context** of fast (start, end, day x of y, etc.)\n* **Length** of fast (8 hours, 3 days, etc.)\n* **Why?** What you hope to accomplish with your fast\n* **Notes** How is it going so far? Any concerns? Insights to share?\n\nBe sure to check back often as comments get posted throughout the day. Sort comments by ""new"" to be sure the newer comments get some love as well.",intermittentfasting
4,"i’ve been IF since 2016. avg 20 ish hours a day. eat for 2-4 hours. repeat.\n\nevery test is done fasted. results always between 100-115.\n\non days i’ve gotten it below 100, 90-99, i’ve fasted for 2-3 days.\n\nwhat is going on? HA1C stays at 5.\n\nthanks for any replies",intermittentfasting
...,...,...
1919,"I've been in recovery for about 3 months, and it's like the weight gain isn't stopping. I don't think I am eating over my daily caloric needs and yet the weight just keeps piling on.

In [24]:
# Count the total number of null values in the "Subreddit" column
null_count = simplified_posts["Subreddit"].isna().sum()

# Print the count
print(f'Total number of null values in the "Subreddit" column: {null_count}')

Total number of null values in the "Subreddit" column: 0


Total number of null values in the "Subreddit" column: 198
